#### Useful link:
https://spacy.io/usage/examples#section-training

In [204]:
import spacy
import pandas as pd
from __future__ import unicode_literals, print_function
import plac
import random
from pathlib import Path
from spacy.util import minibatch, compounding
import os

In [ ]:
def read_datasets():
    return (pd.read_csv('../datasets/TrainNER.csv', delimiter=';', encoding='cp1252'), 
            pd.read_csv('../datasets/Test1NER.csv', names=['Sentence #', 'Word', 'POS', 'Tag'], delimiter=';', encoding='cp1252'), 
            pd.read_csv('../datasets/Test2NER.csv', names=['Sentence #', 'Word', 'POS', 'Tag'], delimiter=';', encoding='cp1252'))

def create_spacy_format_data(df):
    data = []
    for index, row in df.iterrows():
        if index < 1000:
            data.append((str(row['Word']), {'entities': [(0, len(str(row['Word'])), str(row['Tag']))]}))
        else: break
    return data
        
def init_data():
    train, test, test2 = read_datasets()
    print(train.columns)
    print(train.Tag.unique())
    return (create_spacy_format_data(train),
            create_spacy_format_data(test))
#     TEST2_DATA = create_spacy_format_data(test2)


@plac.annotations(
    model=("Model name. Defaults to blank 'en' model.", "option", "m", str),
    output_dir=("Optional output directory", "option", "o", Path),
    n_iter=("Number of training iterations", "option", "n", int))
def main(model=None, output_dir=None, n_iter=100):
    TRAIN_DATA, TEST_DATA = init_data()
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank('en')  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe('ner')

    # add labels
    for _, annotations in TRAIN_DATA:
        for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(n_iter):
            random.shuffle(TRAIN_DATA)
            losses = {}
            # batch up the examples using spaCy's minibatch
            batches = minibatch(TRAIN_DATA, size=compounding(4., 32., 1.001))
            for batch in batches:
                texts, annotations = zip(*batch)
                nlp.update(
                    texts,  # batch of texts
                    annotations,  # batch of annotations
                    drop=0.5,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print('Losses', losses)

    # test the trained model
    print("================TEST==========")
    for text, _ in TEST_DATA:
        doc = nlp(text)
        print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
        print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])

    # save model to output directory
#     if output_dir is not None:
#         output_dir = Path(output_dir)
#         if not output_dir.exists():
#             output_dir.mkdir()
#         nlp.to_disk(output_dir)
#         print("Saved model to", output_dir)

        # test the saved model
#         print("Loading from", output_dir)
#         nlp2 = spacy.load(output_dir)

#     for text, _ in TEST_DATA:
#         doc = nlp(text)
#         print('Entities', [(ent.text, ent.label_) for ent in doc.ents])
#         print('Tokens', [(t.text, t.ent_type_, t.ent_iob) for t in doc])


if __name__ == '__main__':
    main(output_dir='')

Index(['Sentence #', 'Word', 'POS', 'Tag'], dtype='object')
['B-per' 'I-per' 'O' 'B-org' 'B-gpe' 'I-org' 'B-geo' 'I-geo' 'B-time'
 'I-time' 'I-gpe' 'B-obj' 'I-obj' 'B-event' 'I-event' nan]
Created blank 'en' model
Losses {'ner': 22.40148496576722}
Losses {'ner': 15.604965789208665}
Losses {'ner': 13.61118188323601}
Losses {'ner': 13.218861905075453}
Losses {'ner': 10.409557446754034}
Losses {'ner': 8.585881673699134}
Losses {'ner': 7.891694834614505}
Losses {'ner': 6.264976266974879}
Losses {'ner': 6.469181890166241}
Losses {'ner': 6.1273440890608155}
Losses {'ner': 4.867422555473809}
